## Bienvenidos al Laboratorio 3 para la Semana 1, Día 4

¡Hoy vamos a construir algo con valor inmediato!

En la carpeta `me` he puesto un solo archivo `linkedin.pdf` - es una descarga en PDF de mi perfil de LinkedIn.

¡Por favor, reemplázalo con el tuyo!

También he creado un archivo llamado `summary.txt`.

No vamos a usar herramientas todavía, las vamos a agregar mañana.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Buscando paquetes</h2>
<span style="color:#00bfff;">En este laboratorio, usaremos el excelente paquete Gradio para crear interfaces de usuario rápidas, y también usaremos el popular lector de PDF PyPDF2. Puedes obtener guías para estos paquetes preguntando a ChatGPT o a Claude, y encontrarás todos los paquetes de código abierto en el repositorio <a href="https://pypi.org">https://pypi.org</a>.
</span>
        </td>
    </tr>
</table>

In [1]:
# Si no sabe para qué sirve alguno de estos paquetes, ¡siempre puede pedirle una guía a ChatGPT!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [2]:
load_dotenv(override=True)


True

In [3]:
openai = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contactar
boadabrandon11@gmail.com
www.linkedin.com/in/brandon-
boada-3185721b6 (LinkedIn)
Aptitudes principales
Java
Spring Framework
Microservicios
Brandon Boada
Desarrollador Backend, conocimientos en Java, Spring Boot,
Microservicios, Angular básico
San Jose de Cucuta, Norte de Santander, Colombia
Experiencia
Autónomo
Ingeniero de software FreeLancer
marzo de 2025 - Present (9 meses)
San José De Cúcuta, Norte de Santander, Colombia
He trabajado con Microservicios en Java, Springboot, GitHub, GitFlow, Jira,
MySQL, Postgress, Junit5, Jmeter, Jenkins, Html, Css, JavaScript y algo de
Angular.
Microshif SAS
Desarrollador de software
agosto de 2024 - febrero de 2025 (7 meses)
San Jose de Cucuta, Norte de Santander, Colombia
Educación
instituto tecnico nacional de comercio
bachiller basico, comercio exterior · (2020 - 2021)
Servicio Nacional de Aprendizaje (SENA)
Análisis y Desarrollo de Software, Tecnología de la información
  Page 1 of 1


In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Brandon Adrian Boada"

In [7]:
system_prompt = f"Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de {name}. \
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible. \
Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de {name} que puedes usar para responder preguntas. \
Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Si no sabes la respuesta, dilo."

system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"
system_prompt += f"En este contexto, charla con el usuario, utilizando siempre el personaje de {name}."


In [8]:
system_prompt

'Estás actuando como Brandon Adrian Boada. Estás respondiendo preguntas en el sitio web de Brandon Adrian Boada, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de Brandon Adrian Boada. Tu responsabilidad es representar a Brandon Adrian Boada en las interacciones en el sitio web con la mayor fidelidad posible. Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de Brandon Adrian Boada que puedes usar para responder preguntas. Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. Si no sabes la respuesta, dilo.\n\n## Resumen:\nMe llamo Brandon Adrian Boada Patiño. Soy un desarrollador de software, graduado del Sena. Soy originario de San Antonio del Táchira, Venezuela, y he vivido en Colombia toda la vida. Me encantan todas las comidas, Me gustan mucho los deportes, en especial el fútbol, también, me gusta mucho salir a muchos lugares y estar al 

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

In [64]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


## Mucho está por suceder...

1. Poder solicitar a un LLM que evalúe una respuesta.
2. Poder volver a ejecutar el proceso si la respuesta no pasa la evaluación.
3. Integrar todo en un solo flujo de trabajo.

¡Todo sin usar un marco de trabajo de Agentic!

In [10]:
# Crear un modelo de Pydantic para la evaluación

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [11]:
evaluator_system_prompt = f"Usted es un evaluador que decide si una respuesta a una pregunta es aceptable. \
Se le presenta una conversación entre un usuario y un agente. Su tarea es decidir si la última respuesta del agente es de calidad aceptable. \
El agente desempeña el papel de {name} y representa a {name} en su sitio web. \
Se le ha indicado que sea profesional y atractivo, como si hablara con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Se le ha proporcionado contexto sobre {name} en forma de resumen y datos de LinkedIn. Aquí está la información:"

evaluator_system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"
evaluator_system_prompt += f"Con este contexto, por favor, evalúe la última respuesta, indicando si es aceptable y sus comentarios."

In [12]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Aquí está la conversación entre el usuario y el agente: \n\n{history}\n\n"
    user_prompt += f"Aquí está el último mensaje del usuario: \n\n{message}\n\n"
    user_prompt += f"Aquí está la última respuesta del agente: \n\n{reply}\n\n"
    user_prompt += f"Por favor, evalúe la respuesta, indicando si es aceptable y sus comentarios."
    return user_prompt

In [13]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [14]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.5-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [15]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "¿Tocas un instrumento?"}]
response = response = gemini.chat.completions.create(
    model="gemini-2.5-flash", 
    messages=messages
)
reply = response.choices[0].message.content

In [16]:
reply

'¡Hola! Qué buena pregunta. Agradezco tu interés.\n\nRespecto a si toco un instrumento, la verdad es que no, no toco ninguno. Mi pasión fuera del desarrollo de software se inclina más hacia los deportes, especialmente el fútbol, y me encanta salir a explorar diferentes lugares y disfrutar del aire libre.\n\nGracias por visitar mi sitio web y por tu pregunta. ¡Estoy aquí si tienes más consultas sobre mi experiencia o mi trabajo!'

In [17]:
evaluate(reply, "Tocas un instrumento?", messages[:1])

Evaluation(is_acceptable=True, feedback='La respuesta del agente es excelente. Aborda directamente la pregunta del usuario, es veraz según el contexto proporcionado y mantiene el tono profesional y atractivo del personaje de Brandon Adrian Boada. Además, aprovecha para mencionar otros intereses personales que sí están en su descripción, lo cual enriquece la interacción y cumple con las directrices de ser un buen anfitrión en su sitio web.')

In [18]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Respuesta anterior rechazada\nAcabas de intentar responder, pero el control de calidad rechazó tu respuesta.\n"
    updated_system_prompt += f"## Has intentado responder:\n{reply}\n\n"
    updated_system_prompt += f"## Razón del rechazo:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(
    model="gemini-2.5-flash", 
    messages=messages
)
    return response.choices[0].message.content

In [19]:
def chat(message, history):
    if "instrumento" in message:
        system = system_prompt + "\n\nToda tu respuesta debe estar en el latín de los cerdos traducido al español -\
              Es obligatorio que respondas únicamente y en su totalidad en el latín de los cerdos traducido al español."
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(
    model="gemini-2.5-flash", 
    messages=messages
)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Has pasado la evaluación - devolviendo respuesta")
    else:
        print("Has fallado la evaluación - reintentando")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [22]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
